In [1]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [2]:
!pip install sentence-transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.


In [3]:
import openai
import re
# import sklearn

import scipy.special
import itertools

from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

import matplotlib.pyplot as plt
from matplotlib import cm, transforms


import sklearn
import sklearn.metrics
from sklearn.linear_model import LinearRegression
from sklearn.datasets import fetch_20newsgroups

from scipy.stats import wasserstein_distance
from kaggle_secrets import UserSecretsClient
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd
from IPython.display import display


2025-05-03 10:00:02.641585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746266402.943901      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746266403.018396      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# ================== 2. Set OpenAI API Key ==================
user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("OPENAI_API_KEY")

In [5]:
def gpt_query(text):
    res = openai.Completion.create(
        model='gpt-3.5-turbo-instruct',
        prompt=text + "\nRespond with only one word.",
        max_tokens=30,  
        temperature=0
    )
    return res.choices[0].text.strip()


In [6]:
prompt = "What is 36 + 59 ?"
Chat_GPT_Output = gpt_query(prompt)

In [7]:
Chat_GPT_Output


'95'

In [8]:
words = prompt.split()
words

['What', 'is', '36', '+', '59', '?']

In [9]:

seed = 1024
def perturb_text(text_list, perturbation, rng):
    # Select words based on the perturbation flag (1 means include)
    perturbed_list_initial = [item for item, flag in zip(text_list, perturbation) if flag == 1 and item.strip() != ""]
    
    # If no words were selected, force-select at least two words
    if len(perturbed_list_initial) < 2:
        additional_words_needed = 2
        available_words = [item for item in text_list if item.strip() != ""]
        additional_words = rng.choice(available_words, additional_words_needed, replace=False)
        perturbed_list = list(set(perturbed_list_initial + list(additional_words)))
    else:
        perturbed_list = list(set(perturbed_list_initial))
    
    return perturbed_list

# Example text input
text_list = words.copy()

num_perturb = 64  # Number of perturbations to generate
num_words = len(words)

Responses = []
Perturbations = []
unique_perturbations = set()
rng = np.random.default_rng(seed)

# Try up to a certain number of attempts to generate unique perturbations
attempts = 0
max_attempts = num_perturb * 10  # Allow 10 attempts per unique perturbation

# Loop for unique perturbations only
while len(unique_perturbations) < num_perturb and attempts < max_attempts:
    perturbation = tuple(rng.binomial(1, 0.5, size=num_words))
    if perturbation not in unique_perturbations and sum(perturbation) > 0:
        unique_perturbations.add(perturbation)
        perturbed_txt = perturb_text(text_list, perturbation, rng)
        corpus = ' '.join(perturbed_txt)
        Responses.append(corpus)
        Perturbations.append(perturbation)
        print(f"Perturbation: {perturbation}, Perturbed Text: {corpus}")
    attempts += 1

# If more perturbations are needed, allow repeats by sampling from unique_perturbations
while len(Responses) < num_perturb:
    perturbation = rng.choice(list(unique_perturbations))
    perturbed_txt = perturb_text(text_list, perturbation, rng)
    corpus = ' '.join(perturbed_txt)
    Responses.append(corpus)
    Perturbations.append(perturbation)
    print(f"Perturbation (reused): {perturbation}, Perturbed Text: {corpus}")


Perturbation: (0, 0, 0, 1, 1, 0), Perturbed Text: 59 +
Perturbation: (1, 1, 1, 0, 1, 0), Perturbed Text: 59 36 What is
Perturbation: (1, 0, 1, 0, 1, 1), Perturbed Text: ? 59 36 What
Perturbation: (0, 1, 1, 1, 0, 0), Perturbed Text: + 36 is
Perturbation: (1, 0, 0, 0, 0, 1), Perturbed Text: ? What
Perturbation: (1, 1, 1, 1, 1, 1), Perturbed Text: ? is 36 + 59 What
Perturbation: (0, 0, 1, 0, 0, 0), Perturbed Text: + 36
Perturbation: (1, 0, 0, 0, 1, 1), Perturbed Text: 59 ? What
Perturbation: (1, 0, 0, 1, 1, 1), Perturbed Text: 59 + ? What
Perturbation: (1, 0, 1, 1, 1, 0), Perturbed Text: 59 + 36 What
Perturbation: (0, 0, 1, 1, 0, 1), Perturbed Text: + ? 36
Perturbation: (0, 0, 0, 0, 0, 1), Perturbed Text: 36 ? What
Perturbation: (1, 0, 0, 1, 0, 0), Perturbed Text: + What
Perturbation: (1, 0, 1, 1, 1, 1), Perturbed Text: ? 36 + 59 What
Perturbation: (1, 1, 0, 0, 1, 1), Perturbed Text: 59 ? What is
Perturbation: (0, 0, 1, 1, 0, 0), Perturbed Text: + 36
Perturbation: (1, 0, 0, 0, 0, 0), Pert

In [10]:
# Store GPT responses for all perturbations
GPT_Responses = []

# Iterate through the perturbed texts and query GPT
for perturbed_text in Responses:
    print(f"Querying GPT with perturbed text: {perturbed_text}")  # Display the query text
    gpt_response = gpt_query(perturbed_text)  # Get GPT response
    GPT_Responses.append((perturbed_text, gpt_response))  # Save input and output as a tuple

# Display all responses directly in the notebook output
for idx, (input_text, output_text) in enumerate(GPT_Responses):
    print(f"Perturbation {idx + 1}:")
    print(f"GPT Output: {output_text}")
    print("-" * 50)


Querying GPT with perturbed text: 59 +
Querying GPT with perturbed text: 59 36 What is
Querying GPT with perturbed text: ? 59 36 What
Querying GPT with perturbed text: + 36 is
Querying GPT with perturbed text: ? What
Querying GPT with perturbed text: ? is 36 + 59 What
Querying GPT with perturbed text: + 36
Querying GPT with perturbed text: 59 ? What
Querying GPT with perturbed text: 59 + ? What
Querying GPT with perturbed text: 59 + 36 What
Querying GPT with perturbed text: + ? 36
Querying GPT with perturbed text: 36 ? What
Querying GPT with perturbed text: + What
Querying GPT with perturbed text: ? 36 + 59 What
Querying GPT with perturbed text: 59 ? What is
Querying GPT with perturbed text: + 36
Querying GPT with perturbed text: 36 What is
Querying GPT with perturbed text: ? + 36 What
Querying GPT with perturbed text: ? 36 What is
Querying GPT with perturbed text: 59 + What
Querying GPT with perturbed text: 59 + is
Querying GPT with perturbed text: 36 What is
Querying GPT with perturb

In [11]:
from gensim.models import KeyedVectors

In [12]:
!pip install pot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 14.3 MB/s eta 0:00:00


In [13]:
#Word2Vec model
model_path_bin = '/kaggle/input/google_words_/tensorflow2/default/1/GoogleNews-vectors-negative300.bin'
model_txt = KeyedVectors.load_word2vec_format(model_path_bin, binary=True)
model_txt.fill_norms(force=True)


In [14]:
def get_sentence_vector(model, text):
    words = [word for word in clean_text(text).split() if word in model]
    if not words:
        return None
    vectors = [model[word] for word in words]
    return np.mean(vectors, axis=0)


In [15]:
wmd_scores_prompt = []

for perturbed_text in Responses:
    distance = model_txt.wmdistance(prompt.split(), perturbed_text.split())
    wmd_scores_prompt.append((perturbed_text, distance))

print(wmd_scores_prompt)


[('59 +', 0.9073037608020409), ('59 36 What is', 0.45365188040102045), ('? 59 36 What', 0.8783608165937521), ('+ 36 is', 0.43918040829687605), ('? What', 0.8783608165937521), ('? is 36 + 59 What', 0.0), ('+ 36', 0.9073037608020409), ('59 ? What', 0.8783608165937521), ('59 + ? What', 0.437277730007433), ('59 + 36 What', 0.437277730007433), ('+ ? 36', 0.9073037608020409), ('36 ? What', 0.8783608165937521), ('+ What', 0.437277730007433), ('? 36 + 59 What', 0.437277730007433), ('59 ? What is', 0.45365188040102045), ('+ 36', 0.9073037608020409), ('36 What is', 0.45365188040102045), ('? + 36 What', 0.437277730007433), ('? 36 What is', 0.45365188040102045), ('59 + What', 0.437277730007433), ('59 + is', 0.43918040829687605), ('36 What is', 0.45365188040102045), ('59 + What is', 0.0), ('36 What', 0.8783608165937521), ('? is + 59 What', 0.0), ('? What is', 0.45365188040102045), ('+ 36 What is', 0.0), ('+ What is', 0.0), ('? is', 0.874555460014866), ('+ 36 is', 0.43918040829687605), ('36 is', 0.8

In [16]:
def clean_text(text):
    # Remove punctuation and convert to lowercase
    cleaned = re.sub(r'[^\w\s]', '', text.lower())
    # Remove leading/trailing whitespace and newlines
    cleaned = cleaned.strip()
    return cleaned

#  Safe WMD calculation function
def safe_wmdistance(model, text1, text2):
    words1 = [word for word in clean_text(text1).split() if word in model]
    words2 = [word for word in clean_text(text2).split() if word in model]
    if not words1 or not words2:
        return float('nan')
    return model.wmdistance(words1, words2)

import numpy as np

model_txt.fill_norms(force=True)

wmd_scores = []

for perturbed_text, gpt_response in GPT_Responses:
    distance = safe_wmdistance(model_txt, Chat_GPT_Output, gpt_response)
    if np.isnan(distance):
        print(f"Warning: distance is nan for perturbed text: {perturbed_text}")
        distance = 1.0  # assign max distance
    wmd_scores.append((perturbed_text, distance))


#  Print distances
print(wmd_scores)


[('59 +', 1.0), ('59 36 What is', 1.0), ('? 59 36 What', 1.0), ('+ 36 is', 1.0), ('? What', 1.0), ('? is 36 + 59 What', 1.0), ('+ 36', 1.0), ('59 ? What', 1.0), ('59 + ? What', 1.0), ('59 + 36 What', 1.0), ('+ ? 36', 1.0), ('36 ? What', 1.0), ('+ What', 1.0), ('? 36 + 59 What', 1.0), ('59 ? What is', 1.0), ('+ 36', 1.0), ('36 What is', 1.0), ('? + 36 What', 1.0), ('? 36 What is', 1.0), ('59 + What', 1.0), ('59 + is', 1.0), ('36 What is', 1.0), ('59 + What is', 1.0), ('36 What', 1.0), ('? is + 59 What', 1.0), ('? What is', 1.0), ('+ 36 What is', 1.0), ('+ What is', 1.0), ('? is', 1.0), ('+ 36 is', 1.0), ('36 is', 1.0), ('59 + ?', 1.0), ('59 ?', 1.0), ('? 36 What', 1.0), ('? is 36 + 59', 1.0), ('59 36 What', 1.0), ('+ ? is', 1.0), ('59 ? 36', 1.0), ('59 What is', 1.0), ('59 + 36 is', 1.0), ('59 36', 1.0), ('59 + ? 36', 1.0), ('59 + 36', 1.0), ('+ is', 1.0), ('+ 36 What', 1.0), ('59 + ?', 1.0), ('59 What', 1.0), ('59 + is', 1.0), ('? 36', 1.0), ('59 ? is', 1.0), ('? 36 is', 1.0), ('59 ? 3

In [17]:
# Define a small constant to avoid division by zero
epsilon = 1e-8

# Calculate inverse of WMD scores
#inverse_wmd_scores = [(text, 1 / (score + epsilon)) for text, score in wmd_scores]

# Scale inverse WMD scores between 0 and 1 as Similarities
min_value = min(score for _, score in wmd_scores)
max_value = max(score for _, score in wmd_scores)

similarities = [(text, (score - min_value) / (max_value - min_value)) for text, score in wmd_scores]

# Print Similarities
for text, similarity in similarities:
    print(f"Perturbed Text: {text}\nSimilarity Score: {similarity}\n{'-'*50}")

ZeroDivisionError: float division by zero

In [ ]:
# Convert Perturbations to a NumPy array
Perturbations = np.vstack(Perturbations)  # Stack all perturbation vectors vertically

# Convert WMD scores to an array
distance_values = np.array([score for _, score in wmd_scores])

kernel_width = 0.25

# Compute weights based on the distances
weights = np.sqrt(np.exp(-(distance_values ** 2) / (kernel_width ** 2)))
print(weights)
print(Perturbations)

# Fit a Linear Regression model
linear_model = LinearRegression()
linear_model.fit(X=Perturbations, y=[score for _, score in similarities], sample_weight=weights)

# Retrieve coefficients from the model
coefficients = linear_model.coef_

# Print the coefficients and weights
print("Coefficients from the Linear Regression model:", coefficients)



In [ ]:
# Function to plot text heatmap
def plot_text_heatmap(words, scores, title="", width=10, height=0.4, verbose=0, max_word_per_line=20, word_spacing=20, score_fontsize=10, save_path=None):
    fig = plt.figure(figsize=(width, height))
    ax = plt.gca()
    ax.set_title(title, loc='left')
    cmap = plt.cm.ScalarMappable(cmap=plt.cm.bwr)
    cmap.set_clim(0, 1)
    canvas = ax.figure.canvas
    t = ax.transData
    normalized_scores = 0.5 * scores / np.max(np.abs(scores)) + 0.5
    loc_y = -0.2
    for i, (token, score) in enumerate(zip(words, scores)):
        *rgb, _ = cmap.to_rgba(normalized_scores[i], bytes=True)
        color = '#%02x%02x%02x' % tuple(rgb)
        text = ax.text(0.0, loc_y, token, bbox={'facecolor': color, 'pad': 5.0, 'linewidth': 1, 'boxstyle': 'round,pad=0.5'}, transform=t, fontsize=14)
        text.draw(canvas.get_renderer())
        ex = text.get_window_extent()
        score_text = ax.text(0.01, loc_y - 1, f"{score:.2f}", transform=t, fontsize=score_fontsize, ha='center')
        score_text.draw(canvas.get_renderer())
        ex_score = score_text.get_window_extent()
        if (i+1) % max_word_per_line == 0:
            loc_y -= 2.5
            t = ax.transData
        else:
            t = transforms.offset_copy(text._transform, x=ex.width + word_spacing, units='dots')
    if verbose == 0:
        ax.axis('off')
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    plt.show()

# Define the words for heatmap and save path
words = prompt.split(' ')
save_path = '/kaggle/working/text_heatmap.png'

# Call the function with the save path
plot_text_heatmap(words, coefficients, title="Text Heatmap", save_path=save_path)


In [ ]:
y_true = np.array([score for _, score in similarities]).ravel()
y_pred = linear_model.predict(Perturbations).ravel()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
# Calculate regression metrics
wmse = mean_squared_error(y_true, y_pred, sample_weight=weights)
wmae = mean_absolute_error(y_true, y_pred, sample_weight=weights)

# Mean loss (Lm)
mean_loss_f = np.mean(y_true)
mean_loss_g = np.mean(y_pred)
mean_loss = abs(mean_loss_f - mean_loss_g)

# Mean L1 and L2 loss
mean_l1 = np.mean(np.abs(y_true - y_pred))
mean_l2 = np.mean((y_true - y_pred) ** 2)

# Weighted L1 and L2 loss
n = len(y_true)
weighted_l1 = np.sum(weights * np.abs(y_true - y_pred)) / n
weighted_l2 = np.sum(weights * (y_true - y_pred) ** 2) / n

# Weighted R²
f_mean = np.average(y_true, weights=weights)
ss_tot = np.sum(weights * (y_true - f_mean) ** 2)
ss_res = np.sum(weights * (y_true - y_pred) ** 2)
weighted_r2 = 1 - ss_res / ss_tot

# Weighted adjusted R²
p = len(coefficients)
weighted_adj_r2 = 1 - (1 - weighted_r2) * (n - 1) / (n - p - 1)


# Print fidelity metrics
print(100 * '-')
print('Fidelity:')
print(f"Mean Squared Error (MSE): {wmse}")
print(f"Mean Absolute Error (MAE): {wmae}")
print(f"Mean Loss (Lm): {mean_loss}")
print(f"Mean L1 Loss: {mean_l1}")
print(f"Mean L2 Loss: {mean_l2}")
print(f"Weighted L1 Loss: {weighted_l1}")
print(f"Weighted L2 Loss: {weighted_l2}")
print(f"Weighted R-squared (R²ω): {weighted_r2}")
print(f"Weighted Adjusted R-squared (Rˆ²ω): {weighted_adj_r2}")
print(100 * '-')